In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [2]:
df=pd.read_csv('vadu-dummy-data.csv',header=1)
df.head()
unique=np.array([df['Class'].unique()])
print(unique)#found out there are three classes
df=df.drop(columns='SampleID')


[['Vata' 'Kapha' 'Pitta']]


In [3]:
df=df.replace('NA',np.nan)#replaced with actual nan values 
#had to do this else the labelencoder would encode NA as a value as well

In [4]:
# labelencoding on all since i am using random forest model
labels=LabelEncoder()
for i in df.columns:
    df[i]=labels.fit_transform(df[i])
print(df)

     Class  GENDER  F1  F2  F3  F4  ...  F127  F128  F129  F130  F131  F132
0        2       1   1   2   0   0  ...     1     1     1     1     1     0
1        0       1   2   0   0   1  ...     1     1     0     1     0     0
2        2       1   1   2   0   3  ...     0     1     1     0     1     1
3        2       1   1   2   0   2  ...     0     1     1     1     0     1
4        0       1   2   0   0   0  ...     1     1     0     0     0     0
..     ...     ...  ..  ..  ..  ..  ...   ...   ...   ...   ...   ...   ...
126      2       0   1   2   1   3  ...     1     1     1     1     0     1
127      2       0   1   2   1   3  ...     0     0     0     0     0     0
128      2       0   1   2   2   1  ...     1     1     0     0     0     0
129      2       0   1   1   1   3  ...     1     0     1     0     0     1
130      1       0   0   1   2   3  ...     1     0     1     1     0     0

[131 rows x 134 columns]


In [5]:
#replace na values in each row with its mode
for i in df.columns:
    df[i]=df[i].fillna(df[i].mode()[0])

In [6]:
#train test split
X=df.drop(columns='Class')
y=df['Class']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
model_normal=RandomForestClassifier(n_estimators=100,random_state=42)
model_normal.fit(X_train,y_train)
pred=model_normal.predict(X_test)

In [12]:
print(y_test)

55     2
40     1
19     1
31     0
115    2
56     2
69     0
105    2
81     2
26     1
95     2
27     2
64     1
4      0
97     2
100    1
36     0
80     2
93     1
84     1
18     0
10     1
122    2
11     0
127    2
45     2
70     2
Name: Class, dtype: int64


In [16]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(y_test,pred)
print(acc)

0.9259259259259259
